In [ ]:
pip install sentence_transformers

     |████████████████████████████████| 78 kB 5.2 MB/s 
     |████████████████████████████████| 3.5 MB 29.3 MB/s 
     |████████████████████████████████| 6.8 MB 26.5 MB/s 
     |████████████████████████████████| 1.2 MB 33.5 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 29.5 MB/s 
     |████████████████████████████████| 895 kB 49.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=53b25ddcb0b3b07c6a78f691e5047a0afa3c0b93b30639ac31d92abeb813abbf
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from sentence_transformers import SentenceTransformer
import torch

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("gpu num: ", n_gpu)

gpu num:  1


In [ ]:
embedder = SentenceTransformer('bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/630 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import json
from tqdm import tqdm_notebook as tqdm
import pickle

In [ ]:
# this cell to segment my script into scenes 

import os, glob
import re


def segment_screenplay(script_file, new_script_file):
    """
    Find the manual segmentation of the screenplay into scenes and re-write the
    script file with seperation markers. Each scene is separated with a sequence
    of 40 '=' from the previous and next one. Also, before the beginning of each
    scene the scene index (starting from 0) is written between the '=' boundaries.
    :param script_file: Original raw script file for a movie
    :param new_script_file: New segmented script file
    :return: -
    """
    scenes = []
    last_scene = ''
    flag = 0
    with open(script_file, 'r') as f1:
        for line in f1.readlines():
            line_new = re.sub(' +', ' ', line)
            tmp_line = line_new.split()
            tmp_line = [x for x in tmp_line if x != '']
            if ((('INT.' in tmp_line or 'EXT.' in tmp_line or 'INT/EXT.' in tmp_line or
                             'EXT./INT.' in tmp_line or 'INT./EXT.' in tmp_line))):
                flag = 1
                if last_scene != '':
                    scenes.append(last_scene)
                last_scene = line
            else:
                if flag == 1:
                    last_scene += ('\n' + line)

    for i, scene in enumerate(scenes):
        with open(new_script_file, 'a+') as f1:
            f1.write('='*20)
            f1.write(str(i))
            f1.write('='*20)
            f1.write('\n')
            f1.write(scene)
            f1.write('\n')
            f1.write('='*40)
            f1.write('\n')

    return


if __name__ == '__main__':
    screenplays_folder = './Screenplays_and_imdb_meta'
    new_segmented_screenplays = './Segmented_screenplays'

    if not os.path.exists(new_segmented_screenplays):
        os.makedirs(new_segmented_screenplays)

    movie_folder_list = [x[0] for x in os.walk(screenplays_folder)]
    movie_folder_list = glob.glob(screenplays_folder + "/*/")

    for movie in movie_folder_list:
        movie_name = movie.split('/')[-2]
        print(movie_name)
        script_file = os.path.join(movie, movie_name + '_script.txt')
        new_script_file = os.path.join(new_segmented_screenplays,
                                       movie_name + '_script_segmented.txt')
        segment_screenplay(script_file, new_script_file)

In [ ]:
# this cell to get my segmented script 
segment_screenplay('script.txt','segmentscript.txt' )

In [ ]:
# this cell is for when I use the segmented scripts so I divide my dialogue into scenes 
import math 
mylines = []                             # Declare an empty list named mylines.
with open ('segmentscript.txt', 'rt') as myfile: # Open segmentscript.txt for reading text data.
    for myline in myfile:                # For each line, stored as myline,
        mylines.append(myline)           # add its contents to mylines.
mydialogue = []
dialoguetest = []
for i in range(len(mylines)):
    if mylines[i][0] == '=':
      mydialogue.append(dialoguetest)
      dialoguetest =[]
    else :
      dialoguetest.append(mylines[i])

dialogue = []
for i in range(len(mydialogue)):
  test1 = []
  for j in range(len(mydialogue[i])):
    if len(mydialogue[i][j])- len(mydialogue[i][j].lstrip())>9:
        test1.append(mydialogue[i][j])
  dialogue.append(test1)

dialoguetot = []
for i in range(len(dialogue)):
    mydial = ''.join(dialogue[i])                    # In this part what i do is that I put in the good form for the future preprocessing operations
    dial = mydial.split('                 ')    # I take out all the blank spaces and \n and I do a list of list and each list is a sentence said by a new character
    dialogue1 = [s.replace("\n", "") for s in dial]
    dialoguetot.append(dialogue1)

realdial = []
for i in range(len(dialoguetot)):
  if len(dialoguetot[i]) >1 :
    realdial.append(dialoguetot[i])

realdialogue1 = []  #made to take out of dialogue all empty strings
for i in range (len(realdial)):
  conv =[]
  for j in range(len(realdial[i])):
    if j >= 1 :
      conv.append(realdial[i][j])
  realdialogue1.append(conv)



In [ ]:
# This cell is when I keep my entire script and just get all the dialogues into one block 
import math 
mylines = []                             # Declare an empty list named mylines.
with open ('script.txt', 'rt') as myfile: # Open lorem.txt for reading text data.
    for myline in myfile:                # For each line, stored as myline,
        mylines.append(myline)           # add its contents to mylines.
mydialogue = []
for i in range(len(mylines)):
    if len(mylines[i])- len(mylines[i].lstrip())>9:
        mydialogue.append(mylines[i])

mydial = ''.join(mydialogue)                    # In this part what i do is that I put in the good form for the future preprocessing operations
dial = mydial.split('                     ')    # I take out all the blank spaces and \n and I do a list of list and each list is a sentence said by a new character
dialogue = [s.replace("\n", "") for s in dial]
dialoguetot = []
for i in range (len(dialogue)):
    dialogue_i = []
    dialogue_i.append(dialogue[i])
    dialoguetot.append(dialogue_i)

realdialogue = []  #made to take out of dialogue all empty strings
for i in range (len(dialoguetot)):
  if any(c.isalpha() for c in dialoguetot[i][0]) == True:
    realdialogue.append(dialoguetot[i])
  else : 
    pass

# This is to join the Name of the speaker with what it says 
realdial = []
for i in range(len(dialoguetot)):
  if len(dialoguetot[i]) >1 :
    realdial.append(dialoguetot[i])

realdialogue1 = []  #made to take out of dialogue all empty strings
for i in range (len(realdial)):
  conv =[]
  for j in range(len(realdial[i])):
    if j >= 1 :
      conv.append(realdial[i][j])
  realdialogue1.append(conv)

In [ ]:
# this is to group our sentences into group of 10 sentences
conv = [realdialogue[0]]
conv1 = []
for i in range (1, len(realdialogue)):
  if i % 10 == 0:
    conv1.append(realdialogue[i])
    conv.append(conv1)
    conv1 = []
  else :
    conv1.append(realdialogue[i])


In [ ]:
# Test of a new way to divide the movie into three parts as beginning middle end 
import math
n = len(realdialogue)
conv1 = []
conv2 = []
conv3 = []
for i in range(0,math.ceil(n/3)):
  conv1.append(realdialogue[i])
for i in range(math.ceil(n/3), math.ceil((2*n)/3)):
  conv2.append(realdialogue[i])
for i in range(math.ceil((2*n)/3),n):
  conv3.append(realdialogue[i])
conv = [conv1,conv2,conv3]

In [ ]:
# this is to make list of lists because before it was a list of lists of lists 
realconv =[]
for i in range(0, len(conv)):
  flat_conv = []
  for sublist in conv[i]:
    for item in sublist:
        flat_conv.append(item)
  realconv.append(flat_conv)

In [ ]:
def encode_conversation(data, profix):
        
    
    embeddings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(data))):
            #tokens_input = tokenize_conv(sent[i]).cuda()
            embedding = embedder.encode(data[i])

            embeddings.append(embedding)
            
    with open(profix + '_sentence_transformer.pkl', 'wb') as f:
        pickle.dump(embeddings, f)
    
        
    return embeddings

In [ ]:
embeddings = encode_conversation(realdialogue1,'arbitrage')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/110 [00:00<?, ?it/s]